In [13]:
import constants
from config import Config
from utils import batchify, repackage_hidden

import os
import torch
import cPickle as pickle
import random
import numpy as np
from time import time
from math import ceil
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pdb

import torch
from torch.autograd import Variable
from utils import pool_max, pool_avg


## data.py

In [14]:
class BasketConstructor(object):
    '''
        Group products into baskets(type: list)
    '''
    def __init__(self, raw_data_dir, cache_dir):
        self.raw_data_dir = raw_data_dir
        self.cache_dir = cache_dir
    
    def get_orders(self):
        '''
            get order context information
        '''
        orders = pd.read_csv(self.raw_data_dir + 'orders.csv')
        orders = orders.fillna(0.0)
        orders['days'] = orders.groupby(['user_id'])['days_since_prior_order'].cumsum()
        orders['days_last'] = orders.groupby(['user_id'])['days'].transform(max)
        orders['days_up_to_last'] = orders['days_last'] - orders['days']
        del orders['days_last']
        del orders['days']
        return orders
    
    def get_orders_items(self, prior_or_train):
        '''
            get detailed information of prior or train orders 
        '''
        orders_products = pd.read_csv(self.raw_data_dir + 'order_products__%s.csv'%prior_or_train)
        return orders_products
    

    def get_users_orders(self, prior_or_train):
        '''
            get users' prior detailed orders
        '''
        if os.path.exists(self.cache_dir + 'users_orders.pkl'):
            with open(self.cache_dir + 'users_orders.pkl', 'rb') as f:
                users_orders = pickle.load(f)
        else:
            orders = self.get_orders()
            order_products_prior = self.get_orders_items(prior_or_train)
            users_orders = pd.merge(order_products_prior, orders[['user_id', 'order_id', 'order_number', 'days_up_to_last']], 
                        on = ['order_id'], how = 'left')
            with open(self.cache_dir + 'users_orders.pkl', 'wb') as f:
                pickle.dump(users_orders, f, pickle.HIGHEST_PROTOCOL)
        return users_orders
    
    def get_users_products(self, prior_or_train):
        '''
            get users' all purchased products
        '''
        if os.path.exists(self.cache_dir + 'users_products.pkl'):
            with open(self.cache_dir + 'users_products.pkl', 'rb') as f:
                users_products = pickle.load(f)
        else:
            users_products = self.get_users_orders(prior_or_train)[['user_id', 'product_id']].drop_duplicates()
            users_products['product_id'] = users_products.product_id.astype(int)
            users_products['user_id'] = users_products.user_id.astype(int)
            users_products = users_products.groupby(['user_id'])['product_id'].apply(list).reset_index()
            with open(self.cache_dir + 'users_products.pkl', 'wb') as f:
                pickle.dump(users_products, f, pickle.HIGHEST_PROTOCOL)
        return users_products

    def get_items(self, gran):
        '''
            get items' information
            gran = [departments, aisles, products]
        '''
        items = pd.read_csv(self.raw_data_dir + '%s.csv'%gran)
        return items
    
    def get_baskets(self, prior_or_train, reconstruct = False, reordered = False, none_idx = 49689):
        '''
            get users' baskets
        '''
        if reordered:
            filepath = self.cache_dir + './reorder_basket_' + prior_or_train + '.pkl'
        else:
            filepath = self.cache_dir + './basket_' + prior_or_train + '.pkl'
       
        if (not reconstruct) and os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                up_basket = pickle.load(f)
        else:          
            up = self.get_users_orders(prior_or_train).sort_values(['user_id', 'order_number', 'product_id'], ascending = True)
            uid_oid = up[['user_id', 'order_number']].drop_duplicates()
            up = up[up.reordered == 1][['user_id', 'order_number', 'product_id']] if reordered else up[['user_id', 'order_number', 'product_id']]
            # https://stackoverflow.com/questions/41856173/pandas-groupby-list
            up_basket = up.groupby(['user_id', 'order_number'])['product_id'].apply(list).reset_index()
            up_basket = pd.merge(uid_oid, up_basket, on = ['user_id', 'order_number'], how = 'left')
            for row in up_basket.loc[up_basket.product_id.isnull(), 'product_id'].index:
                up_basket.at[row, 'product_id'] = [none_idx]
            up_basket = up_basket.sort_values(['user_id', 'order_number'], ascending = True).groupby(['user_id'])['product_id'].apply(list).reset_index()
            up_basket.columns = ['user_id', 'reorder_basket'] if reordered else ['user_id', 'basket']
            pdb.set_trace()
            with open(filepath, 'wb') as f:
                pickle.dump(up_basket, f, pickle.HIGHEST_PROTOCOL)
        return up_basket
        
    def get_item_history(self, prior_or_train, reconstruct = False, none_idx = 49689):
        filepath = self.cache_dir + './item_history_' + prior_or_train + '.pkl'
        if (not reconstruct) and os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                item_history = pickle.load(f)
        else:
            up = self.get_users_orders(prior_or_train).sort_values(['user_id', 'order_number', 'product_id'], ascending = True)
            item_history = up.groupby(['user_id', 'order_number'])['product_id'].apply(list).reset_index()
            item_history.loc[item_history.order_number == 1, 'product_id'] = item_history.loc[item_history.order_number == 1, 'product_id'] + [none_idx]
            # Do we need this?
            item_history = item_history.sort_values(['user_id', 'order_number'], ascending = True)
            # accumulate 
            item_history['product_id'] = item_history.groupby(['user_id'])['product_id'].transform(pd.Series.cumsum)
            # get unique item list
            item_history['product_id'] = item_history['product_id'].apply(set).apply(list)
            
            # Do we need this?
            item_history = item_history.sort_values(['user_id', 'order_number'], ascending = True)
            # shift each group to make it history
            item_history['product_id'] = item_history.groupby(['user_id'])['product_id'].shift(1)
            for row in item_history.loc[item_history.product_id.isnull(), 'product_id'].index:
                item_history.at[row, 'product_id'] = [none_idx]
            # Is this sufficient?
            item_history = item_history.sort_values(['user_id', 'order_number'], ascending = True).groupby(['user_id'])['product_id'].apply(list).reset_index()
            item_history.columns = ['user_id', 'history_items']

            with open(filepath, 'wb') as f:
                pickle.dump(item_history, f, pickle.HIGHEST_PROTOCOL)
        return item_history       

In [15]:
class Dataset(object):
    '''
        Dataset prepare from user-basket
    '''
    def __init__(self, up_basket, up_r_basket = None, up_his = None):
        if (up_r_basket is not None) and (up_his is not None):
            self.is_reordered_included = True
        else:
            self.is_reordered_included = False

        up_basket['num_baskets'] = up_basket.basket.apply(len)
        self.user_id = list(up_basket.user_id)
        self.num_baskets = [int(n) for n in list(up_basket.num_baskets)]    
        self.basket = [[[int(p) for p in b]for b in u] for u in list(up_basket.basket)]

        if self.is_reordered_included is True:
            up_basket = pd.merge(up_basket, up_r_basket, on = ['user_id'], how = 'left')
            up_basket = pd.merge(up_basket, up_his, on = ['user_id'], how = 'left')
            self.reorder_basket = [[[int(p) for p in b]for b in u] for u in list(up_basket.reorder_basket)]
            self.history_item = [[[int(p) for p in b]for b in u] for u in list(up_basket.history_items)]

    def __getitem__(self, index):
        '''
            return baskets & num_baskets
        '''
        if self.is_reordered_included is True:
            return self.basket[index], self.num_baskets[index], self.user_id[index], self.reorder_basket[index], self.history_item[index]
        else:
            return self.basket[index], self.num_baskets[index], self.user_id[index]

    
    def __len__(self):
        return len(self.user_id)

## Dream.py

In [16]:
class DreamModel(torch.nn.Module):
    '''
       Input Data: b_1, ... b_i ..., b_t
                   b_i stands for user u's ith basket
                   b_i = [p_1,..p_j...,p_n]
                   p_j stands for the  jth product in user u's ith basket
    '''
    def __init__(self, config):
        super(DreamModel, self).__init__()
        # Model configuration
        self.config = config
        # Layer definitons
        self.encode = torch.nn.Embedding(config.num_product, 
                                         config.embedding_dim,
                                         padding_idx = 0) # Item embedding layer
        self.pool = {'avg':pool_avg, 'max':pool_max}[config.basket_pool_type] # Pooling of basket
        # RNN type specify
        if config.rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(torch.nn, config.rnn_type)(config.embedding_dim, 
                                                          config.embedding_dim, 
                                                          config.rnn_layer_num, 
                                                          batch_first=True, 
                                                          dropout=config.dropout)
        else:
            nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[config.rnn_type]
            self.rnn = torch.nn.RNN(config.embedding_dim, 
                                    config.embedding_dim, 
                                    config.rnn_layer_num, 
                                    nonlinearity=nonlinearity, 
                                    batch_first=True, 
                                    dropout=config.dropout)
    
    def forward(self, x, lengths, hidden):
        pdb.set_trace()
        # Basket Encoding 
        ub_seqs = [] # users' basket sequence
        for user in x: # x shape (batch of user, time_step, indice of product) nested lists
            embed_baskets = []
            for basket in user:
                basket = torch.LongTensor(basket).resize_(1, len(basket))
                basket = basket.cuda() if self.config.cuda else basket # use cuda for acceleration
                basket = self.encode(torch.autograd.Variable(basket)) # shape: 1, len(basket), embedding_dim
                embed_baskets.append(self.pool(basket, dim = 1))
            # concat current user's all baskets and append it to users' basket sequence
            try:
                ub_seqs.append(torch.cat(embed_baskets, 1)) # shape: 1, num_basket, embedding_dim
            except:
                pdb.set_trace()
        
        # Input for rnn 
        ub_seqs = torch.cat(ub_seqs, 0).cuda() if self.config.cuda else torch.cat(ub_seqs, 0) # shape: batch_size, max_len, embedding_dim
        packed_ub_seqs = torch.nn.utils.rnn.pack_padded_sequence(ub_seqs, lengths, batch_first=True) # packed sequence as required by pytorch
        
        # RNN
        output, h_u = self.rnn(packed_ub_seqs, hidden)
        dynamic_user, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True) # shape: batch_size, max_len, embedding_dim
        return dynamic_user, h_u
        
    def init_weight(self):
        # Init item embedding
        initrange = 0.1
        self.encode.weight.data.uniform_(-initrange, initrange)
    
    def init_hidden(self, batch_size):
        # Init hidden states for rnn
        weight = next(self.parameters()).data
        if self.config.rnn_type == 'LSTM':
            return (Variable(weight.new(self.config.rnn_layer_num, batch_size, self.config.embedding_dim).zero_()),
                    Variable(weight.new(self.config.rnn_layer_num, batch_size, self.config.embedding_dim).zero_()))
        else:
            return Variable(weight.new(self.config.rnn_layer_num, batch_size, self.config.embedding_dim).zero_())   

## train.py

In [17]:
# CUDA environtments
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0,3,2,1"

# Prepare input
bc = BasketConstructor(constants.RAW_DATA_DIR, constants.FEAT_DATA_DIR)
print("bc done...")
ub_basket = bc.get_baskets('prior', reconstruct = False)
train_ub, test_ub = train_test_split(ub_basket, test_size = 0.2)
del ub_basket
# train_ub, test_ub = Dataset(train_ub), Dataset(test_ub)
dr_config = Config(constants.DREAM_CONFIG)

bc done...


In [20]:
train_ub

,user_id,basket
14678,14679,"[[1541, 2536, 5031, 14788, 17670, 18115, 24838..."
3060,3061,"[[29933], [14631, 23892], [7046, 8152, 23892, ..."
5749,5750,"[[852, 4216, 10957, 12513, 14111, 14325, 23957..."
12707,12708,"[[26968], [327, 5618, 15872, 16759, 17419, 213..."
23285,23286,"[[8671, 12392, 19382, 25069, 25146, 27344, 294..."
17559,17560,"[[2450, 4920, 10644, 21137, 28934, 30190, 3134..."
12762,12763,"[[890, 2846, 9698, 9839, 11125, 30720, 34126, ..."
17133,17134,"[[4210, 5077, 5818, 8424, 12087, 14502, 24622,..."
24079,24080,"[[432, 8309, 13176, 14084, 15991, 16083, 35547..."
12501,12502,"[[4605, 19446, 23909, 24852, 43961], [10840, 1..."


In [6]:
def bpr_loss(x, dynamic_user, item_embedding, config):
    '''
        bayesian personalized ranking loss for implicit feedback
        parameters:
        - x: batch of users' baskets
        - dynamic_user: batch of users' dynamic representations
        - item_embedding: item_embedding matrix
        - config: model configuration
    '''
    nll = 0
    ub_seqs = []
    for u,du in zip(x, dynamic_user):
        du_p_product = torch.mm(du, item_embedding.t()) # shape: max_len, num_item
        nll_u = [] # nll for user
        for t, basket_t in enumerate(u):
            if basket_t[0] != 0 and t != 0:
                pos_idx = torch.cuda.LongTensor(basket_t) if config.cuda else torch.LongTensor(basket_t)
                # Sample negative products
                neg = [random.choice(range(1, config.num_product)) for _ in range(len(basket_t))] # replacement
                # neg = random.sample(range(1, config.num_product), len(basket_t)) # without replacement
                neg_idx = torch.cuda.LongTensor(neg) if config.cuda else torch.LongTensor(neg)
                # Score p(u, t, v > v')
                score = du_p_product[t - 1][pos_idx] - du_p_product[t - 1][neg_idx]
                #Average Negative log likelihood for basket_t
                nll_u.append(- torch.mean(torch.nn.LogSigmoid()(score)))
        nll += torch.mean(torch.cat(nll_u))
    return nll

In [10]:
def train_dream():
    dr_model.train() # turn on training mode for dropout
    dr_hidden = dr_model.init_hidden(dr_config.batch_size) 
    total_loss = 0
    start_time = time()
    num_batchs = ceil(len(train_ub) / dr_config.batch_size)
    pdb.set_trace()
    for i,x in enumerate(batchify(train_ub, dr_config.batch_size)):
        baskets, lens, _ = x
        dr_hidden = repackage_hidden(dr_hidden) # repackage hidden state for RNN
        dr_model.zero_grad() # optim.zero_grad()
        dynamic_user, _  = dr_model(baskets, lens, dr_hidden)
        loss = bpr_loss(baskets, dynamic_user, dr_model.encode.weight, dr_config)
        loss.backward()
        
        # Clip to avoid gradient exploding
        torch.nn.utils.clip_grad_norm(dr_model.parameters(), dr_config.clip) 

        # Parameter updating
        # manual SGD
        # for p in dr_model.parameters(): # Update parameters by -lr*grad
        #    p.data.add_(- dr_config.learning_rate, p.grad.data)
        # adam 
        optim.step()

        total_loss += loss.data
        
        # Logging
        if i % dr_config.log_interval == 0 and i > 0:
            elapsed = (time() - start_time) * 1000 / dr_config.log_interval
            cur_loss = total_loss[0] / dr_config.log_interval # turn tensor into float
            total_loss = 0
            start_time = time()
            print('[Training]| Epochs {:3f} | Batch {:5f} / {:5f} | ms/batch {:02.2f} | Loss {:05.2f} |'.format(epoch, i, num_batchs, elapsed, cur_loss))

In [11]:
def evaluate_dream():
    dr_model.eval()
    dr_hidden = dr_model.init_hidden(dr_config.batch_size) 
    
    total_loss = 0
    start_time = time()
    num_batchs = ceil(len(test_ub) / dr_config.batch_size)
    for i,x in enumerate(batchify(test_ub, dr_config.batch_size)):
        baskets, lens, _ = x
        dynamic_user, _  = dr_model(baskets, lens, dr_hidden)
        loss = bpr_loss(baskets, dynamic_user, dr_model.encode.weight, dr_config)
        dr_hidden = repackage_hidden(dr_hidden)
        total_loss += loss.data
        
    # Logging
    elapsed = (time() - start_time) * 1000 / num_batchs
    total_loss = total_loss[0] / num_batchs
    print('[Evaluation]| Epochs {:3f} | Elapsed {:02.2f} | Loss {:05.2f} |'.format(epoch, elapsed, total_loss))
    return total_loss

In [12]:
dr_model = DreamModel(dr_config)
if dr_config.cuda:
    dr_model.cuda()
print("dr_coonfig done...")
optim = torch.optim.Adam(dr_model.parameters(), lr = dr_config.learning_rate)

best_val_loss = None

try:
    print(dr_config)
    for k,v in constants.DREAM_CONFIG.items():
        print(k,v)
    # training
    for epoch in range(dr_config.epochs):
        train_dream()
        # train_reorder_dream()
        print('-' * 89)
        val_loss = evaluate_dream()
        # val_loss = evaluate_reorder_dream()
        print('-' * 89)
        # checkpoint
        if not best_val_loss or val_loss < best_val_loss:
            with open(dr_config.checkpoint_dir.format(epoch = epoch, loss = val_loss), 'wb') as f:
                torch.save(dr_model, f)
            best_val_loss = val_loss
        else:
            # Manual SGD slow down lr if no improvement in val_loss
            # dr_config.learning_rate = dr_config.learning_rate / 4
            pass
except KeyboardInterrupt:
    print('*' * 89)
    print('Early Stopping!')

dr_coonfig done...
('clip', 20)
('dropout', 0.5)
('batch_size', 4)
('epochs', 100)
('checkpoint_dir', '../Instacart/dream/reorder-next-dream-{epoch:02d}-{loss:.4f}.model')
('basket_pool_type', 'max')
('num_product', 49690)
('embedding_dim', 64)
('log_interval', 1)
('learning_rate', 0.01)
('rnn_layers', 1)
('cuda', False)
('none_idx', 49689)
('rnn_type', 'LSTM')
> <ipython-input-10-c20b6c70a6d0>(8)train_dream()
-> for i,x in enumerate(batchify(train_ub, dr_config.batch_size)):
(Pdb) train_ub.shape()
*** AttributeError: 'Dataset' object has no attribute 'shape'
(Pdb) train_ub.size()
*** AttributeError: 'Dataset' object has no attribute 'size'
(Pdb) len(train_ub)
25600
*****************************************************************************************
Early Stopping!


In [ ]:
val_loss = evaluate_dream()

In [ ]:
ub_basket = bc.get_baskets('prior', reconstruct = False)

In [ ]:
train_ub, test_ub = train_test_split(ub_basket, test_size = 0.001)

In [ ]:
train_ub, test_ub = Dataset(train_ub), Dataset(test_ub)